In [49]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
tickers = ['AAPL', 'BHP.AX']

In [51]:
def RSIcalc(asset):
    df = data.DataReader(
        asset, 
        'yahoo',
        start = '2015/09/01',
        end = '2021/09/01'
    )
    
    df['MA200'] = df['Close'].rolling(window=200).mean()
    df['price change'] = df['Close'].pct_change()
    df['Upmove'] = df['price change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['price change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg Up'] = df['Upmove'].ewm(span=19).mean()
    df['Avg Down'] = df['Downmove'].ewm(span=19).mean()
    df = df.dropna()
    df['RS'] = df['Avg Up'] / df['Avg Down']
    df['RSI'] = df['RS'].apply(lambda x: 100-(100/(x+1)))
    df.loc[(df['Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy'] = 'Yes'
    df.loc[(df['Close'] < df['MA200']) | (df['RSI'] > 30), 'Buy'] = 'No'
    return df




In [52]:
RSIcalc(tickers[0])

High         Low        Open       Close       Volume  \
Date                                                                      
2016-06-15   24.602501   24.257500   24.455000   24.285000  117780800.0   
2016-06-16   24.437500   24.017500   24.112499   24.387501  125307200.0   
2016-06-17   24.162500   23.825001   24.155001   23.832500  244032800.0   
2016-06-20   24.142500   23.757500   24.000000   23.775000  137647600.0   
2016-06-21   24.087500   23.670000   23.735001   23.977501  142185600.0   
...                ...         ...         ...         ...          ...   
2021-08-26  149.119995  147.509995  148.350006  147.539993   48597200.0   
2021-08-27  148.750000  146.830002  147.479996  148.600006   55721500.0   
2021-08-30  153.490005  148.610001  149.000000  153.119995   90956700.0   
2021-08-31  152.800003  151.289993  152.660004  151.830002   86453100.0   
2021-09-01  154.979996  152.339996  152.830002  152.509995   80313700.0   

             Adj Close       MA200  price change    Upmove  Downmove  \
Date                                                                   
2016-06-15   22.652740   26.490925     -0.003283  0.000000  0.003283   
2016-06-16   22.748354   26.471912      0.004221  0.004221  0.000000   
2016-06-17   22.230652   26.456425     -0.022758  0.000000  0.022758   
2016-06-20   22.177019   26.434875     -0.002413  0.000000  0.002413   
2016-06-21   22.365911   26.416800      0.008517  0.008517  0.000000   
...                ...         ...           ...       ...       ...   
2021-08-26  147.539993  131.478400     -0.005527  0.000000  0.005527   
2021-08-27  148.600006  131.641550      0.007185  0.007185  0.000000   
2021-08-30  153.119995  131.809700      0.030417  0.030417  0.000000   
2021-08-31  151.830002  131.972800     -0.008425  0.000000  0.008425   
2021-09-01  152.509995  132.139050      0.004479  0.004479  0.000000   

              Avg Up  Avg Down        RS        RSI Buy  
Date                                                     
2016-06-15  0.003342  0.004138  0.807729  44.681985  No  
2016-06-16  0.003430  0.003724  0.921069  47.945647  No  
2016-06-17  0.003087  0.005627  0.548579  35.424670  No  
2016-06-20  0.002778  0.005306  0.523634  34.367433  No  
2016-06-21  0.003352  0.004775  0.701998  41.245520  No  
...              ...       ...       ...        ...  ..  
2021-08-26  0.003973  0.003948  1.006306  50.157166  No  
2021-08-27  0.004294  0.003553  1.208512  54.720639  No  
2021-08-30  0.006906  0.003198  2.159702  68.351441  No  
2021-08-31  0.006216  0.003720  1.670657  62.556024  No  
2021-09-01  0.006042  0.003348  1.804410  64.341871  No  

[1314 rows x 15 columns]

In [65]:
def getSignals(df):

    Buying_dates = []
    Selling_dates = []
    
    for i in range(len(df) - 11):
        #If column 'Buy'
        if "Yes" in df['Buy'].iloc[i]: 
            Buying_dates.append(df.iloc[i+1].name)
            for j in range(1,11):
                if df['RSI'].iloc[i + j] > 40:
                    Selling_dates.append(df.iloc[i+j].name)
                    break #Break the loop once the condition has been met i.e. RSI > 40
        
                #2nd condition: if 1st condition hasn't been met, sell after maximum 10 days
                elif j == 10:
                    Selling_dates.append(df.iloc[i+j+1].name)   
      
    return Buying_dates, Selling_dates

    

In [54]:
df_test = RSIcalc(tickers[0])


buy, sell = getSignals(df_test)

In [59]:
Profit = (df_test.loc[sell].Close.values - df_test.loc[buy].Close.values) / df_test.loc[buy].Close.values

In [60]:
Profit

array([-0.01433819,  0.00145676,  0.01056598, -0.00380406,  0.01438714,
        0.02688822, -0.00170544,  0.02860752,  0.        ,  0.01540586,
        0.02444366,  0.00711765, -0.02551353, -0.0281908 , -0.0302181 ,
        0.03974696,  0.        ,  0.0058017 ,  0.02104905,  0.09310072,
        0.        ,  0.0153345 ,  0.01946552,  0.05620302,  0.05385122,
        0.        ,  0.        ])

In [61]:
wins = [i for i in Profit if i > 0]

In [62]:
probability = len(wins) / len(Profit)
probability

0.5925925925925926

In [66]:
matrix_signals = []
matrix_profits = []

for i in range(len(tickers)):
    df_test = RSIcalc(tickers[i])
    buy,sell = getSignals(df_test)
    Profit = (df_test.loc[sell].Close.values - df_test.loc[buy].Close.values) / df_test.loc[buy].Close.values
    matrix_signals.append(buy)
    matrix_profits.append(Profit)

In [67]:
matrix_profits

[array([-0.01433819,  0.00145676,  0.01056598, -0.00380406,  0.01438714,
         0.02688822, -0.00170544,  0.02860752,  0.        ,  0.01540586,
         0.02444366,  0.00711765, -0.02551353, -0.0281908 , -0.0302181 ,
         0.03974696,  0.        ,  0.0058017 ,  0.02104905,  0.09310072,
         0.        ,  0.0153345 ,  0.01946552,  0.05620302,  0.05385122,
         0.        ,  0.        ]),
 array([ 0.06125901,  0.0621565 ,  0.05236699,  0.01318852,  0.01987176,
         0.0070385 ,  0.01086961, -0.00268092, -0.03513441, -0.02156332,
        -0.04081633, -0.05630023, -0.05014757, -0.06268576, -0.05566057,
        -0.05344736])]